# Physics 300 
## Computational Physics I (Fall 2017)
## BPB-248, Tues/Thurs 10:00 - 11:15 am 

|Instructor| Prof. Qiang Zhu|
|--|-------------------------------|
|Email | qiang.zhu@unlv.edu|
|Website|http://www.physics.unlv.edu/~qzhu/|
|Office| BPB 232|
|Office hours | Tues/Thurs 8:30 - 10:00 |

# 10 Fast Fourier transform


We have learned how to perform Fourier transform
$$c_k = \sum_{n=0}^{N-1}y_n\exp(-i\frac{2\pi kn}{L})$$

In order to implement the function, we used a for loop to add up the terms in the sum,
and the whole calculation is repeated for each of the $N$ distinct coefficients.
This gives about $N^2$ times for the entire calculation.

This complexity is not bad if we only deal with a few hundreads samples. However, we will always calculated 
mum larger numbers. $O(N^2)$ is not really efficient. 

In [1]:
import numpy as np
def dft1(y):
    '''
    This is a code to do decrete Fourier transform
    '''    
    N = len(y)
    c = np.zeros(N, complex)
    for k in range(N):
        for n in range(N):
            c[k] += y[n]*np.exp(-2j*np.pi*k*n/N)
    return c

In [2]:
%timeit dft1(np.random.random(10))
%timeit dft1(np.random.random(100))
%timeit dft1(np.random.random(200))
%timeit dft1(np.random.random(500))
%timeit dft1(np.random.random(1000))
%timeit dft1(np.random.random(2000))

392 µs ± 37.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
36.6 ms ± 2.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
148 ms ± 7.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
884 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.81 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
15.8 s ± 1.21 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 10.1 how to speed up the calculation?

As we have learned in python, the most important trick is to decrease the number of loops as much as possible.
By looking at the code, it looks like we could use the matrix calculation to replace the 2nd loop when calculating 
$c_k$. The is so called _vectroization_, which is very important when we are programming with high level language
like python and matlab.

In [3]:
def dft2(y):
    '''
    This is a improved code to do DFT
    '''
    x = np.asarray(y, dtype=float)        #N*1 array
    N = y.shape[0]
    n = np.arange(N)                      #1*N array
    k = n.reshape((N, 1))                 #N*1 array
    M = np.exp(-2j * np.pi * k * n / N)   #N*N matrix
    return np.dot(M, x)                   #N*1 array   N*N dot N*1.   N*1

# check if our results agree with the previous code
x = np.random.random(1024)
print(np.allclose(dft1(x), dft2(x)))

True


In [4]:
%timeit dft2(np.random.random(10))
%timeit dft2(np.random.random(100))
%timeit dft2(np.random.random(200))
%timeit dft2(np.random.random(500))
%timeit dft2(np.random.random(1000))
%timeit dft2(np.random.random(2000))

17.3 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
290 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.08 ms ± 7.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.03 ms ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
30.2 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
130 ms ± 4.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Indeed, we immediately find that the new code gets much faster than the first version! 
However, it is still important to know that we might need to deal with 1000000 data and even more. There is a need to improve the code from the fundamental algorithmic aspect. Luckily it turns out that there is a clever trick for calculating DFT much faster. This trick is called `fast Fourier transform (FFT)`, which was originally proposed by Gauss in 1805, and pratically implemented by Cooley and Turkey in 1965.

# 10.2 Fast Fourier transformation
Cooley and Tukey showed that it's possible to divide the DFT computation into two smaller parts. 

From the definition of the DFT we have:

$$\begin{aligned} 
c_{k} & = \sum_{n=0}^{N-1} y_n \cdot \exp(-i \frac{2\pi kn}{N}) \\
      & = \sum_{m=0}^{N/2-1} y_{2m} \cdot \exp(-i 2\pi k \frac{2m}{N}) +  \sum_{m=0}^{N/2-1} y_{2m+1} \exp(-i 2\pi k\frac{2m+1}{N}) \\
      & = \sum_{m=0}^{N/2-1} y_{2m} \cdot \exp(-i 2\pi k \frac{m}{N/2}) + \exp(-i 2\pi k/N) \sum_{m=0}^{N/2-1} y_{2m+1} \exp(-i 2\pi k \frac{m}{N/2})
\end{aligned}$$

Here we split the single DFT into two terms which look very similar to smaller DFT. 
- $A(O, N/2)$, one on the odd numbered values
- $A(E, N/2)$, another on the even numbered values

This doesn't save any cycles, we still need to run $N*N$ cycles.
However, it is now clear that any $c_k$ can be expressed as the summation of $A(E, N/2) + \exp(-i 2\pi k/N)A(O, N/2)$. Therefore, as long as we know $A(E, N/2)$ and $A(O, N/2)$, only $N$ operation is needed.

If we do it iteratively, as long as our smaller Fourier transforms have an even-valued number. 
If $N$=8, we can split into 4, and then 2 and 1. In each cycle, only 8 operations are needed. So the total number of operations is 3\*8 = 24 
In the asymptotic limit, this recursive approach scales as O[$N$log$N$].

In [5]:
#This recursive algorithm can be implemented very quickly in Python, 
#we will call the previous dft until it comes to small numbers:

def fft1(x):
    '''
    A recursive implementation of the 1D Cooley-Tukey FFT    
    '''
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 16:  # this cutoff should be optimized
        return dft2(x)
    else:
        X_even = fft1(x[::2])
        X_odd = fft1(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:int(N/2)] * X_odd,
                               X_even + factor[int(N/2):] * X_odd])

x = np.random.random(1024)    
print(np.allclose(fft1(x), np.fft.fft(x)))


True


In [6]:
#Let's call timeit function to check the speed
x = np.random.random(2048)

%timeit dft1(x)
%timeit dft2(x)
%timeit fft1(x)
%timeit np.fft.fft(x)

15.9 s ± 623 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
135 ms ± 6.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
4.69 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
27.2 µs ± 717 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Now, our fft1 runs much faster than dft2 for large number.
As expected, we still haven't come close to the speed of the built-in FFT algorithm in numpy. 
The FFT algorithm behind numpy's fft is a Fortran implementation which has received years of tweaks and optimizations. Furthermore, our NumPy solution involves both Python-stack recursions and the allocation of many temporary arrays, which adds significant computation time.

## 10.4 Homework
Choose a particular application and perform fft analysis, and illustrate how fft helps.

## 10.5 Acknowledgement
The lecture note was partially inspired by https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/